In [4]:
# import libraries
import pandas as pd
import torch
import torch.nn as neuralnet
import torch.nn.functional as func
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision import transforms

from torch.utils.tensor

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "c:\users\beegu\appdata\local\programs\python\python37\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.